我们在这章讨论字典和集合，因为它们背后都是哈希表，下面是本章的大纲

- 常用字典方法
- 特别处理遗失的键
- 在标准库中，dict 的变化
- set 与 frozenset 形态
- 哈希表的工作原理
- 哈希表的影响（键形态限制，无法预知的排序等等）

## 什么是可哈希化

如果一个对象有一个哈希值，而且在生命周期中不被改变（它需要实现一个 `__hash__()` 方法），而且可以与其它对象比较（需要实现 `__eq__()` 方法），就是可哈希化的。看下面例子

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [5]:
t1 = (1, 2, [30, 40]) # 其中列表是可变的，所以没有哈希值
hash(t1)

TypeError: unhashable type: 'list'

In [6]:
tf = (1, 2, frozenset([30, 40])) #frozenset 是冻结的集合，不可变的，所以有哈希值
hash(tf)

-4118419923444501110

## 构建字典方法

In [8]:
a = dict(one = 1, two = 2, three = 3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

除了常规语法以及 dict 构建之外，我们可以使用字典生成式来构建字典，dictcomp 会由任何一个可迭代对象产生一对 key:value 来构建 dict，下面是使用字典生成式的一个例子:

In [11]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States')
]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1}

字典有一个内置方法 `d.update(m, [**kargs])` 它会先判断 m，如果 m 有 keys 方法，就假定它是个映射，负责迭代 m，假设它的元素是 (key, value)，这也就是说明任何产生 (key, value) 的迭代对象都可以构建字典

## 使用 setdefault 处理找不到的键

当 dict 使用 d[k] 时，发现 k 不是现有键时，会跑出 KeyError 错误，我们知道 d.get(k, default) 是 d[k] 的另一种用法，使用预处理值要比处理 KeyError 异常更方便，但是如果你要更新你找到的值，使用 `__getitem__()` 和 `get()` 都很没有效率，我们下面先看一下一般时候处理找不到键的方式:

In [15]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re

WORD_RE = re.compile('\w+') # \w 是匹配任意字母或数字，+ 是匹配一次到任意次

index = {}
#with open(sys.argv[1], encoding="utf-8") as fp:  #正常文件名是参数传的
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    # line_no 是索引（从 1 开始），line 是行的内容
        for match in WORD_RE.finditer(line):  # 返回所有匹配子串，返回类型是迭代器
            word = match.group()              # group 获取该单词 （match 是一个对象）
            column_no = match.start() + 1     # 获取列数，索引从 0 开始
            location = (line_no, column_no)   # 构造一个元组，内容是 (row, col)
            # 这样写很糟糕，这里仅仅是为了演示
            occurrences = index.get(word, []) # 判断该单词是否被添加过，没有返回 [ ]，注意返回的是原列表的一个备份
            occurrences.append(location)      # 为该 key 对应的值添加内容
            index[word] = occurrences         # 这要搜索 word 这个 key 第二次
for word in sorted(index, key = str.upper):   # 按照字母顺序排序，忽略大小写
    print(word, index[word])        

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


我们处理 occurrences 的三行可以使用 dict.setdefault 来改为一行

In [16]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re

WORD_RE = re.compile('\w+') # \w 是匹配任意字母或数字，+ 是匹配一次到任意次

index = {}
#with open(sys.argv[1], encoding="utf-8") as fp:  #正常文件名是参数传的
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    # line_no 是索引（从 1 开始），line 是行的内容
        for match in WORD_RE.finditer(line):  # 返回所有匹配子串，返回类型是迭代器
            word = match.group()              # group 获取该单词 （match 是一个对象）
            column_no = match.start() + 1     # 获取列数，索引从 0 开始
            location = (line_no, column_no)   # 构造一个元组，内容是 (row, col)
            index.setdefault(word, []).append(location) #如果没有 word 这个 key，设为 [ ]，setdefault 会传回该值，所以不用二次搜索就可以被更新
for word in sorted(index, key = str.upper):   # 按照字母顺序排序，忽略大小写
    print(word, index[word])      

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


换句话说 `index.setdefault(word, []).append(location)` 与下面等价

In [18]:
#if key not in my_dict:
#    my_dict[key] = []
#else:
#    my_dict[key].append(new_value)

## 可弹性查找键的映射

有时在找不到键时，返回一些虚构值还是很好的，这里有两种方法，第一种是使用预设字典而不是普通字典，第二种是将字典变成子类别或其它的映射类型，并添加一个 `__missing__` 方法，下面会讨论两种做法

### defaultdict 找不到键的另一种做法

下面是使用 collections.defaultdict 优雅的解决上面搜索不存在的键的问题。

In [20]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re
import collections

WORD_RE = re.compile('\w+') 

index = collections.defaultdict(list)  # 使用 list 建立 defaultdict，将它当成 default_factory
#with open(sys.argv[1], encoding="utf-8") as fp:  
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    
        for match in WORD_RE.finditer(line):  
            word = match.group()              
            column_no = match.start() + 1    
            location = (line_no, column_no)  
            # 如果不存在 word 键，会调用初始化传的 default_factory 产生一个预设值, 如果没有指定 default_factory，会产生 KeyError 异常
            index[word].append(location) 
for word in sorted(index, key = str.upper):  
    print(word, index[word])      

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


工作原理: 当我们初始化一个 defaultdict 时候，要提供一个方法，当 `__getitem__()` 找不到键的时候，会用它产生一个预设值，这里我们将 list 传进去，每次调用会产生一个空列表。

注意：defaultdict 的 default_factory 只是为了提供一个预设值，不供其它方法使用，例如 dd 是 defaultdict，k 是不存在的键， dd[k] 会调用 default_factory 产生一个预设值，而 dd.get(k) 仍然传回 None，下面是例子

In [23]:
import collections

index = collections.defaultdict(list) 
print(index.get('hello'))

None


调用 default_factory 的 defaultdict 机制，实际上是调用 `__missing__()` 特殊方法，这是所有变暗映射类型都有的方法

## `__missing__` 方法

映射处理找不到键时，底层使用的是 `__missing__()` 方法，基础的 dict 并没有这个方法，但是如果你将 dict 变成子类别，并提供一个 `__missing__()` 方法，标准的 `dict.__getitem__()`  会在找不到键时调用它，而非发出 KeyError

`__missing__()` 只被 `__getitem__()` 调用（即 d[k] 运算符），`__missing__()` 方法的存在，并不会影响到其它方法查询键的行为，例如 get 或 `__contains__`（in 运算符），这就是 defaultdict 只能与 `__getitem__()` 一起使用的原因

如果你想要一个映射，它的键在查询时会被转成 str 

In [25]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
    
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'